In [1]:
# testing reaction_recommender class
import collections
import itertools
import libsbml
import networkx as nx
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import requests

import matplotlib.pyplot as plt
%matplotlib inline  

script_path = "/Users/woosubs/Desktop/AutomateAnnotation/reaction_recommender/reaction_recommender/"
os.sys.path.append(script_path)

# import get_annotation as ga
from reaction_recommender import annotation_container as ac
from reaction_recommender import reaction_recommender as recommender

BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
BIOMD_12 = 'BIOMD0000000012.xml'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
# BioModel files
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
biomd_files = [f for f in os.listdir(BIOMODEL_DIR) if f[-4:] == ".xml"]
print(len(biomd_files))
biomd_fpaths = [os.path.join(BIOMODEL_DIR, one_file) for one_file in biomd_files]

# BiGG files
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")
bigg_files = [f for f in os.listdir(BIGG_DIR) if f[-4:]=='.xml']
bigg_fpaths = [os.path.join(BIGG_DIR, f) for f in bigg_files]
print(len(bigg_files))
one_fpath = bigg_fpaths[0]

1000
108


In [2]:
os.sys.path

['/Users/woosubs/Desktop/AutomateAnnotation/reaction_recommender/reaction_recommender/notebook',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python38.zip',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python3.8',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python3.8/lib-dynload',
 '',
 '/Users/woosubs/.local/lib/python3.8/site-packages',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python3.8/site-packages',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python3.8/site-packages/IPython/extensions',
 '/Users/woosubs/.ipython',
 '/Users/woosubs/anaconda3/envs/pyo/lib/python3.8/site-packages/libsbml',
 '/Users/woosubs/Desktop/AutomateAnnotation/reaction_recommender/reaction_recommender/']

In [3]:
# Nov 14, 2021: using a shortened version, meaning H and D are removed
with open(os.path.join(DATA_DIR, 'all_shortened_chebi_to_formula_19nov2021.pkl'), 'rb') as f:
  ref_shortened_chebi_to_formula = pickle.load(f)
# below is full version
with open(os.path.join(DATA_DIR, 'all_possible_chebi_to_formula_12nov2021.pkl'), 'rb') as f:
  ref_full_chebi_to_formula = pickle.load(f)
# map secondary id to primary id
with open(os.path.join(DATA_DIR, 'chebi_second2prime_8nov2021.pickle'), 'rb') as f:
  ref_second2prime_dict = pickle.load(f)
# RHEA All Ids to Bi Id (KEGG Mappable)
with open(os.path.join(RHEA_DIR, 'rhea_all2bi.pkl'), 'rb') as f:
  ref_rhea2bi = pickle.load(f)
with open(os.path.join(RHEA_DIR, 'rhea_all2master.pkl'), 'rb') as f:
  ref_rhea2master = pickle.load(f)
with open(os.path.join(RHEA_DIR, 'rhea2chebi_reference.pkl'), 'rb') as f:
  ref_rhea2chebi = pickle.load(f)

In [4]:
one_container = ac.AnnotationContainer(model_file=os.path.join(BIGG_DIR, ecoli_fpath))

In [5]:
reader = libsbml.SBMLReader()
document = reader.readSBML(ecoli_fpath)
model = document.getModel()

In [6]:
one_spec_anot = model.getSpecies('M_glc__D_e').getAnnotationString()
one_spec_anot

'<sbml:annotation xmlns:sbml="http://www.sbml.org/sbml/level3/version1/core">\n  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">\n    <rdf:Description rdf:about="#M_glc__D_e">\n      <bqbiol:is xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/bigg.metabolite/glc__D"/>\n          <rdf:li rdf:resource="http://identifiers.org/biocyc/META:Glucopyranose"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:12965"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:20999"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:4167"/>\n          <rdf:li rdf:resource="http://identifiers.org/chebi/CHEBI:17634"/>\n          <rdf:li rdf:resource="http://identifiers.org/hmdb/HMDB00122"/>\n          <rdf:li rdf:resource="http://identifiers.org/hmdb/HMDB06564"/>\n          <rdf:li rdf:resource="http://identifiers.org/inchi_key/WQZGKKKJIJFFOK-GASJEMHNS

In [7]:
one_container.getOntologyFromString(one_spec_anot)

[('bigg.metabolite', 'glc__D'),
 ('biocyc', 'META:Glucopyranose'),
 ('chebi', 'CHEBI:12965'),
 ('chebi', 'CHEBI:20999'),
 ('chebi', 'CHEBI:4167'),
 ('chebi', 'CHEBI:17634'),
 ('hmdb', 'HMDB00122'),
 ('hmdb', 'HMDB06564'),
 ('inchi_key', 'WQZGKKKJIJFFOK-GASJEMHNSA-N'),
 ('kegg.compound', 'C00031'),
 ('kegg.drug', 'D00009'),
 ('metanetx.chemical', 'MNXM41'),
 ('sabiork', '1406'),
 ('sabiork', '1407'),
 ('seed.compound', 'cpd26821'),
 ('seed.compound', 'cpd00027')]

In [8]:
recom = recommender.ReactionRecommender(model_file=os.path.join(BIGG_DIR, ecoli_fpath))

In [9]:
recom.ref_mat

,C9N2O7PS,C21O6,C6N4O4,C34N6O21P,O3S2R,C9N3O2S,C11N3O6S,C17N3O10PSR,C6ClO3,C24N7O17P3SR,...,(C6O5)n.C6H12O6,C12O3S,C18O6S,BO4,C111N7O41P2,C6FO3P,(C12NO19S3)n.C6H13NO8S,C36N3O27R,C50N13O12,C17NO3R
RHEA:10003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:10019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RHEA:67147,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:67151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:67155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RHEA:67159,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
recommender.ref_short_rhea2formula
dummy_dict = {'RHEA:10003': ['N', 'O', 'C5NO', 'C5O2'],
              'RHEA:10007': ['C8NS', 'C8NS']}
dummy_mat = recom.getReferenceMatrix(dummy_dict)
dummy_mat

,N,C5NO,C5O2,O,C8NS
RHEA:10003,1,1,1,1,0
RHEA:10007,0,0,0,0,1


In [11]:
dummy_mat.loc['RHEA:10003', 'N']

1

In [22]:
recom.getCHEBIToFormula(id_to_chebis={'a': ['CHEBI:15379']})

{'a': ['O2']}

In [24]:
recom.spec_formula_dict['M_o2_c']

['O2']

In [15]:
ref_second2prime_dict

{'CHEBI:18357': 'CHEBI:18357',
 'CHEBI:1': 'CHEBI:18357',
 'CHEBI:14668': 'CHEBI:18357',
 'CHEBI:25592': 'CHEBI:18357',
 'CHEBI:258884': 'CHEBI:18357',
 'CHEBI:43725': 'CHEBI:18357',
 'CHEBI:10': 'CHEBI:10',
 'CHEBI:133004': 'CHEBI:133004',
 'CHEBI:100': 'CHEBI:100',
 'CHEBI:16114': 'CHEBI:16114',
 'CHEBI:14576': 'CHEBI:16114',
 'CHEBI:18492': 'CHEBI:16114',
 'CHEBI:25177': 'CHEBI:16114',
 'CHEBI:27497': 'CHEBI:16114',
 'CHEBI:76924': 'CHEBI:76924',
 'CHEBI:75766': 'CHEBI:76924',
 'CHEBI:76925': 'CHEBI:76924',
 'CHEBI:6714': 'CHEBI:6714',
 'CHEBI:16713': 'CHEBI:16713',
 'CHEBI:1000': 'CHEBI:16713',
 'CHEBI:11508': 'CHEBI:16713',
 'CHEBI:19453': 'CHEBI:16713',
 'CHEBI:10000': 'CHEBI:10000',
 'CHEBI:46955': 'CHEBI:46955',
 'CHEBI:100000': 'CHEBI:100000',
 'CHEBI:22712': 'CHEBI:22712',
 'CHEBI:36820': 'CHEBI:36820',
 'CHEBI:38777': 'CHEBI:38777',
 'CHEBI:100001': 'CHEBI:100001',
 'CHEBI:20857': 'CHEBI:20857',
 'CHEBI:100002': 'CHEBI:100002',
 'CHEBI:24995': 'CHEBI:24995',
 'CHEBI:52898': 